# PC4 (ENTREGABLE 1)

## **Proyecto 5: Desplegar una API de preguntas y respuestas basada en LLM finamente ajustado**

### **Entregable 1 - Fecha: 30 de noviembre**

**Trabajo (8 puntos)**

| Criterio                                                                | Puntos |
|-------------------------------------------------------------------------|--------|
| Ajuste fino del LLM para preguntas y respuestas                         | 3      |
| Indexación y organización eficiente de la base de datos de documentos   | 3      |
| Implementación inicial de búsqueda eficiente en los documentos          | 2      |
| **Total**                                                               | **8**  |

**Exposición (12 puntos)**

| Criterio                                                                 | Puntos |
|--------------------------------------------------------------------------|--------|
| Descripción del LLM y su ajuste para la tarea específica                 | 4      |
| Explicación de la base de datos y mecanismos de búsqueda implementados   | 4      |
| Claridad en la presentación y capacidad para responder preguntas         | 4      |
| **Total**                                                                | **12** |

In [ ]:
!nvidia-smi

Tue Nov 26 01:00:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
pip install transformers fastapi uvicorn elasticsearch sentence-transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.1/543.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does no

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments, pipeline
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader, RandomSampler
from tqdm import tqdm
from transformers.data.processors.squad import SquadProcessor
from transformers import squad_convert_examples_to_features

## SQUAD

Se importa dataset SQUAD para el entrenamiento de BERT basado en preguntas y respuestas

In [ ]:
dataset = load_dataset("squad")
# dataset["train"] = dataset["train"].select(range(1000))
# dataset["validation"] = dataset["validation"].select(range(100))

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Estructura de los datos de entrenamiento

In [ ]:
dataset["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

Se define una función para obtener la siguiente estructura:

```
{
  input_ids:
  tokenized_question and answer:
  answer_start_pos:
  answer_end_pos:
}
```


Se usará el modelo base de BERT `distilbert base-uncased`
https://huggingface.co/docs/transformers/model_doc/distilbert

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments


MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


def preprocess_function_with_labels(examples):
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        truncation=True,  #trunca los inputs con una longitud max_length
        padding="max_length",   #modifica la longitud de los inputs para que sea igual a max_length
        max_length=384,
        stride=128,   #se segmenta inputs mayores a max_length de manera que se conserven los 'stride' tokens anteriores
        return_overflowing_tokens=True,
        return_offsets_mapping=True,    #retorna los mapeos entre posiciones de caracteres y tokens
        return_tensors="pt",    #formato pytorch
    )

    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]

    start_positions = []
    end_positions = []

    #para cada offset mapeado entre tokens y caracteres de las respuestas
    for i, offsets in enumerate(offset_mapping):

        sample_idx = sample_mapping[i]
        answer = answers[sample_idx]

        #Si no hay respuesta
        if len(answer["answer_start"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:

            start_char = answer["answer_start"][0]
            end_char = start_char + len(answer["text"][0])

            #Tokens de inicio y fin de la respuesta dentro del contexto
            token_start = 0
            token_end = 0
            for idx, (start, end) in enumerate(offsets):
                if start <= start_char < end:
                    token_start = idx
                if start < end_char <= end:
                    token_end = idx
                    break

            start_positions.append(token_start)
            end_positions.append(token_end)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

#Se aplica el preprocesamiento a todo el dataset
tokenized_datasets = dataset.map(
    preprocess_function_with_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
from pprint import pp
pp(tokenized_datasets["train"].features)
print(len(tokenized_datasets["train"]))

{'input_ids': Sequence(feature=Value(dtype='int32', id=None),
                       length=-1,
                       id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None),
                            length=-1,
                            id=None),
 'start_positions': Value(dtype='int64', id=None),
 'end_positions': Value(dtype='int64', id=None)}
88524


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Entrenamiento del modelo con el dataset transformado

In [ ]:
import os


os.environ["WANDB_DISABLED"] = "true"

model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)
model = model.to(device)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=1000,
    logging_dir="./logs",
    logging_steps=500,
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)


trainer.train()


model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-12-863e97018de0>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.450200,1.379402
2,1.152100,1.305569
3,0.960100,1.341003


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [ ]:
def answer_question(question, context):

    inputs = tokenizer.encode_plus(
        question,
        context,
        add_special_tokens=True,
        return_tensors="pt"
    ).to(device)


    input_ids = inputs["input_ids"].tolist()[0]
    outputs = model(**inputs)


    answer_start = outputs.start_logits.argmax()
    answer_end = outputs.end_logits.argmax() + 1


    answer = tokenizer.decode(input_ids[answer_start:answer_end])
    return answer

In [ ]:
question = "Who created BERT?"
context = (
    "Bidirectional encoder representations from transformers BERT is a language model introduced in October 2018 by researchers at Google. It learns to represent text as a sequence of vectors using self-supervised learning. It uses the encoder-only transformer architecture. It is notable for its dramatic improvement over previous state-of-the-art models, and as an early example of a large language model"
)

# Get the answer
answer = answer_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")

NameError: name 'answer_question' is not defined

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("MrKite/bert_squad")
model

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
     

In [ ]:
#Importa modelo guardado
tokenizer = AutoTokenizer.from_pretrained("MrKite/bert_squad")
tokenizer

DistilBertTokenizerFast(name_or_path='MrKite/bert_squad', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
qa_pipeline = pipeline("question-answering", model="MrKite/bert_squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
qa_pipeline(question=question, context=context)

{'score': 0.8081843852996826,
 'start': 111,
 'end': 132,
 'answer': 'researchers at Google'}

In [ ]:
print(torch.__version__)

2.5.1+cu121


In [ ]:
tokenizer.save_pretrained("finetunedtokenizer", safe_serialization=False)

('finetunedtokenizer/tokenizer_config.json',
 'finetunedtokenizer/special_tokens_map.json',
 'finetunedtokenizer/vocab.txt',
 'finetunedtokenizer/added_tokens.json',
 'finetunedtokenizer/tokenizer.json')

## Database

Base de datos de libros de ciencia e ingenieria:
https://huggingface.co/datasets/open-phi/textbooks

In [ ]:
#Dataset de textos de libros de ciencias
db = load_dataset("open-phi/textbooks")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

(…)-00000-of-00001-b513d9e388d56453.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1795 [00:00<?, ? examples/s]

In [ ]:
from pprint import pp
pp(db["train"][0]['outline'])

('# Table of Contents\n'
 '- [Computational Cognitive Science: A Comprehensive '
 'Guide":](#Computational-Cognitive-Science:-A-Comprehensive-Guide":)\n'
 '  - [Foreward](#Foreward)\n'
 '  - [Chapter 1: Introduction and Organizational '
 'Meeting](#Chapter-1:-Introduction-and-Organizational-Meeting)\n'
 '    - [Introduction](#Introduction)\n'
 '    - [Section: 1.1 Course Overview](#Section:-1.1-Course-Overview)\n'
 '      - [Subsection: 1.1.1 Course '
 'Structure](#Subsection:-1.1.1-Course-Structure)\n'
 '      - [Subsection: 1.1.2 Key Concepts and '
 'Terminologies](#Subsection:-1.1.2-Key-Concepts-and-Terminologies)\n'
 '      - [Subsection: 1.1.3 Course Goals](#Subsection:-1.1.3-Course-Goals)\n'
 '    - [Section: 1.2 Administrative '
 'details](#Section:-1.2-Administrative-details)\n'
 '      - [Subsection: 1.2.1 Course '
 'Administration](#Subsection:-1.2.1-Course-Administration)\n'
 '      - [Subsection: 1.2.2 Course '
 'Materials](#Subsection:-1.2.2-Course-Materials)\n'
 '      - 

In [ ]:
!pip install elasticsearch

### Elasticsearch

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import ConnectionError

es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"], timeout=30, max_retries=10, retry_on_timeout=True)

try:
    health = es.cluster.health()
    print("Elasticsearch Cluster Health:", health)

    version = es.info()
    print("Elasticsearch Version:", version['version']['number'])


    doc = {
        'author': 'John Doe',
        'text': 'Elasticsearch is fun!',
        'timestamp': '2024-11-26'
    }

    response = es.index(index="test_index", id=1, document=doc)
    print("Document indexed:", response)

    retrieved_doc = es.get(index="test_index", id=1)
    print("Retrieved Document:", retrieved_doc['_source'])

except ConnectionError:
    print("Error: Could not connect to Elasticsearch. Please make sure the server is running.")
except Exception as e:
    print("An error occurred:", e)


<ipython-input-20-0c8ba992aea6>:5: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"], timeout=30, max_retries=10, retry_on_timeout=True)


Elasticsearch Cluster Health: {'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 0, 'active_shards': 0, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}
Elasticsearch Version: 8.15.5
Document indexed: {'_index': 'test_index', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
Retrieved Document: {'author': 'John Doe', 'text': 'Elasticsearch is fun!', 'timestamp': '2024-11-26'}


In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import ConnectionError


es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"], timeout=30, max_retries=10, retry_on_timeout=True)

# Check if Elasticsearch is running
try:
    # Get the cluster health (status of the Elasticsearch cluster)
    health = es.cluster.health()
    print("Elasticsearch Cluster Health:", health)

    # Check the Elasticsearch version
    version = es.info()
    print("Elasticsearch Version:", version['version']['number'])

    # Index a document
    for id, doc in enumerate(db["train"]):
        response = es.index(index="textbooks", id=id, document=doc)
        print("Document indexed:", response)

except ConnectionError:
    print("Error: Could not connect to Elasticsearch. Please make sure the server is running.")
except Exception as e:
    print("An error occurred:", e)


<ipython-input-28-583908fdc50c>:5: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"], timeout=30, max_retries=10, retry_on_timeout=True)


Elasticsearch Cluster Health: {'cluster_name': 'docker-cluster', 'status': 'yellow', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 1, 'active_shards': 1, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 1, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 50.0}
Elasticsearch Version: 8.15.5
Document indexed: {'_index': 'textbooks', '_id': '0', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
Document indexed: {'_index': 'textbooks', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
Document indexed: {'_index': 'textbooks', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_p

In [ ]:
response = es.count(index="textbooks")
print("Document count in test_index:", response['count'])

Document count in test_index: 1795


In [ ]:
from pprint import pp

response = es.indices.get_mapping(index="textbooks")
pp(response.body)

{'textbooks': {'mappings': {'properties': {'concepts': {'type': 'text',
                                                        'fields': {'keyword': {'type': 'keyword',
                                                                               'ignore_above': 256}}},
                                           'field': {'type': 'text',
                                                     'fields': {'keyword': {'type': 'keyword',
                                                                            'ignore_above': 256}}},
                                           'markdown': {'type': 'text',
                                                        'fields': {'keyword': {'type': 'keyword',
                                                                               'ignore_above': 256}}},
                                           'model': {'type': 'text',
                                                     'fields': {'keyword': {'type': 'keyword',
                          

In [ ]:
response = es.indices.get_field_mapping(index="textbooks", fields="concepts")
print("Field Mapping for 'text':")
pp(response.body)

Field Mapping for 'text':
{'textbooks': {'mappings': {'concepts': {'full_name': 'concepts',
                                         'mapping': {'concepts': {'type': 'text',
                                                                  'fields': {'keyword': {'type': 'keyword',
                                                                                         'ignore_above': 256}}}}}}}}


In [ ]:
from elasticsearch import Elasticsearch
from pprint import pp

index_name="textbooks"
# Create an Elasticsearch client
es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"])  # Adjust the host as needed

response = es.indices.put_settings(
    index=index_name,
    body={
        "index": {
            "highlight.max_analyzed_offset": 2000000
        }
    }
)
settings = es.indices.get_settings(index=index_name)
print(settings)


{'textbooks': {'settings': {'index': {'highlight': {'max_analyzed_offset': '2000000'}, 'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'textbooks', 'creation_date': '1732675700945', 'number_of_replicas': '1', 'uuid': 'b5Z7LztsRIGxQc0nd_n-FA', 'version': {'created': '8512000'}}}}}


### Highlighting
https://www.elastic.co/guide/en/elasticsearch/reference/current/highlighting.html

In [ ]:
from elasticsearch import Elasticsearch
from pprint import pp
index_name="textbooks"

# Create an Elasticsearch client
es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"])  # Adjust the host as needed

response = es.indices.put_settings(
    index=index_name,
    body={
        "index": {
            "highlight.max_analyzed_offset": 2000000
        }
    }
)

# Define the search query
query = {
    "query": {
        "match": {
            "markdown": "What is Archimedes' principle"
        },

    },
    "highlight": {
        "fields": {
            "markdown": {
                "fragment_size": 500,  # Smaller fragment size
                "number_of_fragments": 3
            }
        }
    },
}

# Perform the search on the "test_index" index
response = es.search(index="textbooks", body=query)

# Print results with highlighting
print(len(response["hits"]["hits"]))
for hit in response["hits"]["hits"][:min(5, len(response["hits"]["hits"]))]:
    print("-"*20)
    print(f"Document ID: {hit['_id']}")
    print(f"Score: {hit['_score']}")
    hit["_source"]["outline"] = ["Indices"]
    hit["_source"]["markdown"] = ["Texto de libro"]
    pp(hit["_source"])

    print("Matched Text Fragments with Highlighting:")
    for fragment in hit["highlight"]["markdown"]:
        print("-"*10)
        print(fragment)
        m = fragment.index("<em>")
        # print(fragment[min(0, m-50):max(len(fragment), m+50)])


10
--------------------
Document ID: 102
Score: 12.145523
{'topic': 'Principles of Naval Architecture.',
 'model': 'gpt-4-0613',
 'concepts': '[]',
 'outline': ['Indices'],
 'markdown': ['Texto de libro'],
 'field': 'mechanical_engineering',
 'subfield': 'fluid_mechanics',
 'rag': 'N/A'}
Matched Text Fragments with Highlighting:
----------
In the next subsection, we will delve deeper into <em>Archimedes</em>' <em>Principle</em> and its application in naval architecture.



#### 2.2b <em>Archimedes</em>' <em>Principle</em>



<em>Archimedes</em>' <em>Principle</em> is a fundamental law of physics that governs the behavior of objects submerged in fluids. It states that "Any object, wholly or partially immersed in a fluid, is buoyed up by a force equal to the weight of the fluid displaced by the object."
----------
This <em>principle</em> is the foundation of the concept of buoyancy and is of paramount importance in naval architecture.



To understand <em>Archimedes</em>' <em>Principle</

In [ ]:
from elasticsearch import Elasticsearch
from pprint import pp
index_name="textbooks"

# Create an Elasticsearch client
es = Elasticsearch(["https://mrkite-elasticsearch.hf.space"])  # Adjust the host as needed

response = es.indices.put_settings(
    index=index_name,
    body={
        "index": {
            "highlight.max_analyzed_offset": 2000000
        }
    }
)

# Define the search query
query = {
    "query": {
        "match": {
            "markdown": "Archimedes' principle"
        },

    },
    "highlight": {
        "type": "plain",
        "fields": {
            "markdown": {
                "fragment_size": 500,  # Smaller fragment size
                "number_of_fragments": 3
            }
        }
    },
}

# Perform the search on the "test_index" index
response = es.search(index="textbooks", body=query)

# Print results with highlighting
print(len(response["hits"]["hits"]))
for hit in response["hits"]["hits"][:min(5, len(response["hits"]["hits"]))]:
    print("-"*20)
    print(f"Document ID: {hit['_id']}")
    print(f"Score: {hit['_score']}")
    hit["_source"]["outline"] = ["Indices"]
    hit["_source"]["markdown"] = ["Texto de libro"]
    pp(hit["_source"])

    print("Matched Text Fragments with Highlighting:")
    for fragment in hit["highlight"]["markdown"]:
        print("-"*10)
        print(fragment)
        m = fragment.index("<em>")
        # print(fragment[min(0, m-50):max(len(fragment), m+50)])


10
--------------------
Document ID: 102
Score: 12.145523
{'topic': 'Principles of Naval Architecture.',
 'model': 'gpt-4-0613',
 'concepts': '[]',
 'outline': ['Indices'],
 'markdown': ['Texto de libro'],
 'field': 'mechanical_engineering',
 'subfield': 'fluid_mechanics',
 'rag': 'N/A'}
Matched Text Fragments with Highlighting:
----------
 <em>Archimedes</em>' <em>principle</em> and the equation of hydrostatic equilibrium, will be presented and explained. For instance, <em>Archimedes</em>' <em>principle</em>, which can be expressed as `$\Delta w = \rho_f \cdot g \cdot V$`, where `$\Delta w$` is the weight of the displaced fluid, `$\rho_f$` is the density of the fluid, `$g$` is the acceleration due to gravity, and `$V$` is the volume of the displaced fluid.



By the end of this chapter, readers should have a solid understanding of the principles of hydrostatics and their
----------
$` is the height of the fluid column above the point in question. This equation tells us that the pressu